In [ ]:
import pandas as pd
df = pd.read_csv('personality_datasert.csv')
df.head()

,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,4.0,No,4.0,6.0,No,13.0,5.0,Extrovert
1,9.0,Yes,0.0,0.0,Yes,0.0,3.0,Introvert
2,9.0,Yes,1.0,2.0,Yes,5.0,2.0,Introvert
3,0.0,No,6.0,7.0,No,14.0,8.0,Extrovert
4,3.0,No,9.0,4.0,No,8.0,5.0,Extrovert


In [2]:
yes_no = {'Yes':1, 'No':0}
personality = {'Extrovert': 1, 'Introvert': 0}

In [3]:
df = df.replace(to_replace={'Stage_fear': yes_no, 'Drained_after_socializing': yes_no, 'Personality': personality})

df.head()

C:\Users\admin\AppData\Local\Temp\ipykernel_14964\3503400377.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(to_replace={'Stage_fear': yes_no, 'Drained_after_socializing': yes_no, 'Personality': personality})


,Time_spent_Alone,Stage_fear,Social_event_attendance,Going_outside,Drained_after_socializing,Friends_circle_size,Post_frequency,Personality
0,4.0,0,4.0,6.0,0,13.0,5.0,1
1,9.0,1,0.0,0.0,1,0.0,3.0,0
2,9.0,1,1.0,2.0,1,5.0,2.0,0
3,0.0,0,6.0,7.0,0,14.0,8.0,1
4,3.0,0,9.0,4.0,0,8.0,5.0,1


In [4]:
# 학습 데이터, 타겟 데이터 
personality_target = df['Personality'].to_numpy()
personality_data = df.drop('Personality', axis=1).to_numpy()

In [6]:
personality_target[:10]

array([1, 0, 0, 1, 1, 1, 1, 1, 0, 1])

In [8]:
# 훈련세트,  테스트 세트 
from sklearn.model_selection import train_test_split, GridSearchCV
train_input, test_input, train_target, test_target = train_test_split(
    personality_data, personality_target, test_size=0.1, random_state=42
)

In [9]:
# 최적의 모델을 찾기!
from sklearn.ensemble import HistGradientBoostingClassifier
import numpy as np

params = {
    'max_iter': np.arange(90, 300, 10),
    'learning_rate': [0.0001, 0.001, 0.002, 0.003, 0.004, 0.005, 0.01],
}
gs = GridSearchCV(HistGradientBoostingClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

,estimator,HistGradientB...ndom_state=42)
,param_grid,"{'learning_rate': [0.0001, 0.001, ...], 'max_iter': array([ 90, 1...70, 280, 290])}"
,scoring,None
,n_jobs,-1
,refit,True
,cv,None
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,loss,'log_loss'


In [10]:
np.mean(gs.cv_results_['mean_test_score'])

np.float64(0.875087966220971)

In [11]:
gs.best_params_

{'learning_rate': 0.01, 'max_iter': np.int64(220)}

In [12]:
hgb = gs.best_estimator_
print("학습세트:", hgb.score(train_input, train_target))
print("테스트세트:", hgb.score(test_input, test_target))

학습세트: 0.9352490421455939
테스트세트: 0.9275862068965517


In [14]:
# 학습한 최적의 모델을 저장
import pickle
with open("model.pkl", "wb") as f:
    pickle.dump(hgb, f)

In [17]:
test_input[:5].tolist()

[[10.0, 1.0, 3.0, 3.0, 1.0, 5.0, 3.0],
 [4.505816002819881, 0.0, 3.963354474982382, 5.0, 0.0, 14.0, 5.0],
 [5.0, 1.0, 0.0, 0.0, 1.0, 0.0, 2.0],
 [10.0, 1.0, 2.0, 2.0, 1.0, 2.0, 1.0],
 [7.0, 1.0, 0.0, 0.0, 1.0, 2.0, 1.0]]